In [1]:
import torch
import torch.nn as nn

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)

    def forward(self, x):
        return self.embedding(x)

# Example
vocab_size = 10000  # Number of unique tokens
embed_size = 512    # Size of each embedding vector
embedding_layer = TokenEmbedding(vocab_size, embed_size)
sample_input = torch.randint(0, vocab_size, (2, 5))  # Batch of 2, sequence length 5
print(embedding_layer(sample_input))

tensor([[[-0.5107,  1.2109,  0.4443,  ..., -1.1329, -1.0040, -0.8407],
         [ 1.4061, -0.3158, -0.1791,  ..., -0.6145, -1.2156, -1.3990],
         [-0.1380, -1.1112,  1.5099,  ..., -0.1847,  1.0982,  0.9937],
         [-0.9383, -0.0634, -1.7327,  ..., -1.6067,  0.1475, -0.7512],
         [ 0.3003, -0.7274, -0.8942,  ...,  0.7736, -0.7878,  0.1172]],

        [[ 0.6407,  0.1711, -0.2416,  ..., -0.2125, -1.3315,  0.3198],
         [ 0.3834, -0.2596,  0.3608,  ..., -0.2569,  0.2668,  0.5353],
         [-0.9457, -0.2280,  0.3347,  ..., -0.0652,  0.7554, -0.2658],
         [ 0.3577, -1.4637, -1.0498,  ..., -0.4715,  0.2167, -0.4580],
         [ 0.7452,  1.2762,  0.6175,  ..., -0.9870, -0.9253, -1.4866]]],
       grad_fn=<EmbeddingBackward0>)


In [2]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, max_len=5000):
        super(PositionalEncoding, self).__init__()
        positional_encoding = torch.zeros(max_len, embed_size)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, embed_size, 2).float() * (-math.log(10000.0) / embed_size))
        positional_encoding[:, 0::2] = torch.sin(position * div_term)
        positional_encoding[:, 1::2] = torch.cos(position * div_term)
        self.positional_encoding = positional_encoding.unsqueeze(0)

    def forward(self, x):
        return x + self.positional_encoding[:, :x.size(1), :]

# Example
pos_encoding = PositionalEncoding(embed_size)
sample_embed = torch.randn(2, 5, embed_size)  # Batch of 2, sequence length 5
print(pos_encoding(sample_embed))

tensor([[[-1.3631,  1.1541,  0.6658,  ...,  0.0752, -1.1213,  1.4952],
         [ 1.2312, -1.4363,  1.0039,  ..., -0.7289, -1.2935,  2.5008],
         [ 1.6639, -0.8617,  1.1879,  ...,  0.4361,  1.3306,  0.1026],
         [-0.8595, -1.5329, -0.8286,  ...,  0.2086, -0.2765, -0.2055],
         [-0.0370,  0.6262, -0.3793,  ...,  1.2921, -1.3632,  0.7509]],

        [[ 0.1019,  1.4819, -1.2683,  ...,  1.4860, -0.6222,  2.4296],
         [ 1.4432,  1.7550,  2.1792,  ...,  0.9714, -2.0048,  1.6655],
         [ 1.3472, -1.6747,  0.9007,  ...,  2.1774, -0.8418,  2.9598],
         [-2.1297,  1.1564,  1.1838,  ...,  1.4510, -0.4971,  1.1330],
         [ 0.5640, -1.7029, -0.1998,  ..., -0.1671, -0.8699, -1.0074]]])


In [3]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert self.head_dim * heads == embed_size, "Embed size must be divisible by heads"

        self.values = nn.Linear(embed_size, embed_size)
        self.keys = nn.Linear(embed_size, embed_size)
        self.queries = nn.Linear(embed_size, embed_size)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, queries, mask):
        N = queries.shape[0]  # Batch size
        value_len, key_len, query_len = values.shape[1], keys.shape[1], queries.shape[1]

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        # Split embeddings into multiple heads
        values = values.view(N, value_len, self.heads, self.head_dim).transpose(1, 2)
        keys = keys.view(N, key_len, self.heads, self.head_dim).transpose(1, 2)
        queries = queries.view(N, query_len, self.heads, self.head_dim).transpose(1, 2)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])  # Dot product
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(N, query_len, self.embed_size)
        return self.fc_out(out)

# Example
self_attention = SelfAttention(embed_size, heads=8)
sample_values = torch.randn(2, 5, embed_size)
sample_keys = torch.randn(2, 5, embed_size)
sample_queries = torch.randn(2, 5, embed_size)
print(self_attention(sample_values, sample_keys, sample_queries, mask=None))

tensor([[[-0.2650, -0.0138, -0.1369,  ...,  0.0160, -0.1675, -0.1164],
         [-0.0399,  0.0829,  0.0787,  ...,  0.1724,  0.0318, -0.0610],
         [-0.1632,  0.0071,  0.0424,  ...,  0.0222,  0.0265, -0.1099],
         [-0.1177, -0.0558, -0.2181,  ...,  0.0786, -0.1171,  0.2296],
         [-0.1284,  0.0700, -0.0907,  ..., -0.1487, -0.0528, -0.1558]],

        [[-0.1644, -0.1165, -0.0481,  ...,  0.1379, -0.0786, -0.1489],
         [ 0.0566,  0.0702,  0.0818,  ..., -0.0725, -0.1113,  0.1477],
         [-0.0342,  0.0040,  0.0669,  ...,  0.0646, -0.0468,  0.0088],
         [-0.2835,  0.2084, -0.2218,  ...,  0.0305,  0.1727,  0.1536],
         [ 0.3001,  0.0468, -0.0181,  ...,  0.0645, -0.1630, -0.0014]]],
       grad_fn=<ViewBackward0>)


In [4]:
class FeedForward(nn.Module):
    def __init__(self, embed_size, hidden_dim):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(embed_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, embed_size)

    def forward(self, x):
        return self.fc2(torch.relu(self.fc1(x)))

# Example
ffn = FeedForward(embed_size, hidden_dim=2048)
sample_input = torch.randn(2, 5, embed_size)
print(ffn(sample_input))

tensor([[[ 0.2502,  0.4043,  0.2476,  ..., -0.2305, -0.0144,  0.3515],
         [ 0.1349,  0.0393, -0.2413,  ..., -0.0292, -0.0358,  0.0553],
         [ 0.3929, -0.2188,  0.0845,  ..., -0.0292, -0.0968,  0.0571],
         [ 0.0740, -0.2844,  0.3542,  ...,  0.0218,  0.0322, -0.2568],
         [ 0.1023,  0.2264,  0.3761,  ..., -0.0058, -0.0663,  0.2831]],

        [[ 0.1933, -0.3518, -0.0547,  ..., -0.0491,  0.0792,  0.2080],
         [-0.1730,  0.1200,  0.3326,  ...,  0.2827, -0.0876,  0.3738],
         [ 0.1477,  0.0287,  0.0007,  ..., -0.1772, -0.4980,  0.4286],
         [-0.1465,  0.3344,  0.1029,  ..., -0.3432,  0.1370,  0.3417],
         [ 0.1065, -0.2235,  0.2269,  ...,  0.0145,  0.0438,  0.1827]]],
       grad_fn=<ViewBackward0>)


In [5]:
class LayerNorm(nn.Module):
    def __init__(self, embed_size, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(embed_size))
        self.beta = nn.Parameter(torch.zeros(embed_size))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

# Example
layer_norm = LayerNorm(embed_size)
sample_input = torch.randn(2, 5, embed_size)
print(layer_norm(sample_input))

tensor([[[ 0.1283, -0.2381, -2.0961,  ...,  1.5004, -1.4037, -1.0683],
         [-1.0427, -0.0933, -0.3127,  ..., -1.1825, -1.9134,  0.2614],
         [ 0.6290, -0.1767, -0.5160,  ..., -1.0853, -0.8685, -1.1553],
         [ 0.8093, -0.9181, -0.5589,  ..., -0.1668,  0.5268,  0.9670],
         [ 0.0094, -1.0394, -1.1688,  ..., -0.7448,  0.0141,  0.4391]],

        [[-0.0940, -1.9512,  0.5669,  ..., -0.7354, -0.7408, -0.3125],
         [ 1.4880,  0.7455,  0.7313,  ..., -0.8094,  0.0893,  0.0118],
         [ 0.5296, -1.3687, -0.5897,  ...,  1.3760,  0.5349,  0.9400],
         [ 0.6803, -0.3986, -0.1089,  ..., -1.1432, -2.5175, -1.8575],
         [-0.4143, -0.0696, -0.7616,  ...,  0.2276, -0.9817,  1.6380]]],
       grad_fn=<AddBackward0>)


In [6]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, hidden_dim, dropout):
        super(TransformerBlock, self).__init__()
        self.self_attention = SelfAttention(embed_size, heads)
        self.norm1 = LayerNorm(embed_size)
        self.norm2 = LayerNorm(embed_size)
        self.feed_forward = FeedForward(embed_size, hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.self_attention(value, key, query, mask)
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

# Example
transformer_block = TransformerBlock(embed_size, heads=8, hidden_dim=2048, dropout=0.1)
sample_values = torch.randn(2, 5, embed_size)
print(transformer_block(sample_values, sample_values, sample_values, mask=None))

tensor([[[-1.0407, -0.3299,  0.0000,  ..., -0.8929, -1.0715,  0.0000],
         [-1.1608,  0.9155,  1.0867,  ..., -0.0986, -0.5380, -0.6693],
         [ 0.0000, -0.8146,  0.3248,  ..., -0.2282,  1.3188, -1.4808],
         [-0.6809,  1.2364,  0.5343,  ...,  1.9219,  1.4572,  0.2046],
         [-0.6255,  0.7176,  0.3445,  ..., -1.2273, -0.3527, -1.2008]],

        [[-0.4991, -1.1902,  1.7805,  ..., -1.6173,  1.2536,  1.1088],
         [ 0.1819,  0.4296,  1.7259,  ...,  0.7301,  1.4286, -0.8408],
         [-0.1110, -2.3102,  0.7448,  ...,  2.0697, -0.1144,  1.4206],
         [ 0.2557, -1.0940,  0.4652,  ..., -0.5974,  0.8166,  0.6838],
         [ 0.2001,  0.5810,  1.4826,  ...,  2.4731,  0.1476,  0.6006]]],
       grad_fn=<MulBackward0>)
